# Using PyTorch to Generate Spongebob Transcripts

1. import libraries we will depend on.

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
from argparse import Namespace

2. Set variables for this project

In [2]:
flags = Namespace(
    train_file='../web_scraper/spongebob-transcript.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['_________________________________________________________'],
    predict_top_k=5,
    checkpoint_path='checkpoint'
)

3. Define a function to process the raw data

In [3]:
def process_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r') as file:
        text = file.read()
    text = text.split()
    
    word_counts = Counter(text)
    
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)
    
    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

#Call the function and set some variables
device = torch.device('cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = process_data_from_file(flags.train_file, flags.batch_size, flags.seq_size)

4. Take a look at some of the data

In [4]:
print('Vocabulary size:\n', n_vocab)
print('Out Text Matrix:\n', out_text)

Vocabulary size:
 71566
Out Text Matrix:
 [[12942   265 23161 ...   543   227    72]
 [  647    82   196 ...     1    89    30]
 [    8     3  4628 ...   188    70 15392]
 ...
 [ 2404    22    76 ... 11386    29  8081]
 [ 1060   131     0 ...  4740     5 14194]
 [ 7903  4002   203 ...   182   626  2083]]


Looks good. Now let's:
5. Create the network in Pytorch

In [5]:
class RNNModule(nn.Module):
    #define necessary layers in the constructor:
    #embedding layer, LSTM layer, dense layer
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size, lstm_size, batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
    
    #define function for forward pass
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)
        return logits, state
    
    #define function to reset state at each epoch
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

#Instantiate the network
net = RNNModule(n_vocab, flags.seq_size, flags.embedding_size, flags.lstm_size)
net = net.to(device)

6. Define a function to handle loss and training

In [6]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    return criterion, optimizer

#Call the function
criterion, optimizer = get_loss_and_train_op(net, 0.01)

7. Define the function to get batches for training

In [7]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

8. Define the prediction function which will be used in training

In [8]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    # tells the network we are about to evaluate
    net.eval()
    
    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])
    
    # append another word
    words.append(int_to_vocab[choice])
    
    # append 100 more
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
        
        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])
    
    print(' '.join(words))
    

8. Training! Loop through batches for each epoch, compute losses and update network parameters

In [ ]:
iteration = 0
for e in range(50):
    print('Epoch: ', e)
    batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
    state_h, state_c = net.zero_state(flags.batch_size)
    
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for x, y in batches:
        iteration += 1
        print('Iteration: ', iteration)
        
        # train
        net.train()
        
        # reset gradients
        optimizer.zero_grad()
        
        x = torch.tensor(x).to(device)
        y = torch.tensor(y).to(device)
            
        logits, (state_h, state_c) = net(x, (state_h, state_c))
        loss = criterion(logits.transpose(1,2), y)
        
        # detach() so pytorch can calculate loss
        state_h = state_h.detach()
        state_c = state_c.detach()
        
        loss_value = loss.item()
        
        # back propagation
        loss.backward()
        
        # gradient clipping 
        _ = torch.nn.utils.clip_grad_norm_(net.parameters(), flags.gradients_norm)
        
        # update network parameters
        optimizer.step()
        
        # print loss values to the console during training
        if iteration % 100 == 0:
            print('Epoch: {}/{}'.format(e, 200),
                  'Iteration: {}'.format(iteration),
                  'Loss: {}'.format(loss_value))
        
        # print a little sample of text during training
        if iteration % 1000 == 0:
            predict(device, net, flags.initial_words, n_vocab, vocab_to_int, int_to_vocab, top_k=5)
            torch.save(net.state_dict(), '../checkpoint_pt/model-{}-{}.pth'.format(iteration, loss_value))

Epoch:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Iteration

Iteration:  503
Iteration:  504
Iteration:  505
Iteration:  506
Iteration:  507
Iteration:  508
Iteration:  509
Iteration:  510
Iteration:  511
Iteration:  512
Iteration:  513
Iteration:  514
Iteration:  515
Iteration:  516
Iteration:  517
Iteration:  518
Iteration:  519
Iteration:  520
Iteration:  521
Iteration:  522
Iteration:  523
Iteration:  524
Iteration:  525
Iteration:  526
Iteration:  527
Iteration:  528
Iteration:  529
Iteration:  530
Iteration:  531
Iteration:  532
Iteration:  533
Iteration:  534
Iteration:  535
Iteration:  536
Iteration:  537
Iteration:  538
Iteration:  539
Iteration:  540
Iteration:  541
Iteration:  542
Iteration:  543
Iteration:  544
Iteration:  545
Iteration:  546
Iteration:  547
Iteration:  548
Iteration:  549
Iteration:  550
Iteration:  551
Iteration:  552
Iteration:  553
Iteration:  554
Iteration:  555
Iteration:  556
Iteration:  557
Iteration:  558
Iteration:  559
Iteration:  560
Iteration:  561
Iteration:  562
Iteration:  563
Iteration:  564
Iteratio

_________________________________________________________ ha! I got an idea. Squidward: I have an idea and I don't want you to be my friend, I'm gonna do that. Squidward: I need to be a new Krabby Patties on my old idea! Squidward: Hey, that's a good time for your thumbs I got you for you and I can't be my old little clown. I don't know you have to be a good thing to go in the door is a little ice cream is in my new best friend I got a little idea! Mr. Krabs: Oh, that's you have a good to get to get
Iteration:  1001
Iteration:  1002
Iteration:  1003
Iteration:  1004
Iteration:  1005
Iteration:  1006
Iteration:  1007
Iteration:  1008
Iteration:  1009
Iteration:  1010
Iteration:  1011
Iteration:  1012
Iteration:  1013
Iteration:  1014
Iteration:  1015
Iteration:  1016
Iteration:  1017
Iteration:  1018
Iteration:  1019
Iteration:  1020
Iteration:  1021
Iteration:  1022
Iteration:  1023
Iteration:  1024
Iteration:  1025
Iteration:  1026
Iteration:  1027
Iteration:  1028
Iteration:  1029
It

Iteration:  1441
Iteration:  1442
Iteration:  1443
Iteration:  1444
Iteration:  1445
Iteration:  1446
Iteration:  1447
Iteration:  1448
Iteration:  1449
Iteration:  1450
Iteration:  1451
Iteration:  1452
Iteration:  1453
Iteration:  1454
Iteration:  1455
Iteration:  1456
Iteration:  1457
Iteration:  1458
Iteration:  1459
Iteration:  1460
Iteration:  1461
Iteration:  1462
Iteration:  1463
Iteration:  1464
Iteration:  1465
Iteration:  1466
Iteration:  1467
Iteration:  1468
Iteration:  1469
Iteration:  1470
Iteration:  1471
Iteration:  1472
Iteration:  1473
Iteration:  1474
Iteration:  1475
Iteration:  1476
Iteration:  1477
Iteration:  1478
Iteration:  1479
Iteration:  1480
Iteration:  1481
Iteration:  1482
Iteration:  1483
Iteration:  1484
Iteration:  1485
Iteration:  1486
Iteration:  1487
Iteration:  1488
Iteration:  1489
Iteration:  1490
Iteration:  1491
Iteration:  1492
Iteration:  1493
Iteration:  1494
Iteration:  1495
Iteration:  1496
Iteration:  1497
Iteration:  1498
Iteration:  14

Iteration:  1908
Iteration:  1909
Iteration:  1910
Iteration:  1911
Iteration:  1912
Iteration:  1913
Iteration:  1914
Iteration:  1915
Iteration:  1916
Iteration:  1917
Iteration:  1918
Iteration:  1919
Iteration:  1920
Iteration:  1921
Iteration:  1922
Iteration:  1923
Iteration:  1924
Iteration:  1925
Iteration:  1926
Iteration:  1927
Iteration:  1928
Iteration:  1929
Iteration:  1930
Iteration:  1931
Iteration:  1932
Iteration:  1933
Iteration:  1934
Iteration:  1935
Iteration:  1936
Iteration:  1937
Iteration:  1938
Iteration:  1939
Iteration:  1940
Iteration:  1941
Iteration:  1942
Iteration:  1943
Iteration:  1944
Iteration:  1945
Iteration:  1946
Iteration:  1947
Iteration:  1948
Iteration:  1949
Iteration:  1950
Iteration:  1951
Iteration:  1952
Iteration:  1953
Iteration:  1954
Iteration:  1955
Iteration:  1956
Iteration:  1957
Iteration:  1958
Iteration:  1959
Iteration:  1960
Iteration:  1961
Iteration:  1962
Iteration:  1963
Iteration:  1964
Iteration:  1965
Iteration:  19

...

Iteration:  7691
Iteration:  7692
Iteration:  7693
Iteration:  7694
Iteration:  7695
Iteration:  7696
Iteration:  7697
Iteration:  7698
Iteration:  7699
Iteration:  7700
Epoch: 3/200 Iteration: 7700 Loss: 4.889597415924072
Iteration:  7701
Iteration:  7702
Iteration:  7703
Iteration:  7704
Iteration:  7705
Iteration:  7706
Iteration:  7707
Iteration:  7708
Iteration:  7709
Iteration:  7710
Iteration:  7711
Iteration:  7712
Iteration:  7713
Iteration:  7714
Iteration:  7715
Iteration:  7716
Iteration:  7717
Iteration:  7718
Iteration:  7719
Iteration:  7720
Iteration:  7721
Iteration:  7722
Iteration:  7723
Iteration:  7724
Iteration:  7725
Iteration:  7726
Iteration:  7727
Iteration:  7728
Iteration:  7729
Iteration:  7730
Iteration:  7731
Iteration:  7732
Iteration:  7733
Iteration:  7734
Iteration:  7735
Iteration:  7736
Iteration:  7737
Iteration:  7738
Iteration:  7739
Iteration:  7740
Iteration:  7741
Iteration:  7742
Iteration:  7743
Iteration:  7744
Iteration:  7745
Iteration:  

Iteration:  8002
Iteration:  8003
Iteration:  8004
Iteration:  8005
Iteration:  8006
Iteration:  8007
Iteration:  8008
Iteration:  8009
Iteration:  8010
Iteration:  8011
Iteration:  8012
Iteration:  8013
Iteration:  8014
Iteration:  8015
Iteration:  8016
Iteration:  8017
Iteration:  8018
Iteration:  8019
Iteration:  8020
Iteration:  8021
Iteration:  8022
Iteration:  8023
Iteration:  8024
Iteration:  8025
Iteration:  8026
Iteration:  8027
Iteration:  8028
Iteration:  8029
Iteration:  8030
Iteration:  8031
Iteration:  8032
Iteration:  8033
Iteration:  8034
Iteration:  8035
Iteration:  8036
Iteration:  8037
Iteration:  8038
Iteration:  8039
Iteration:  8040
Iteration:  8041
Iteration:  8042
Iteration:  8043
Iteration:  8044
Iteration:  8045
Iteration:  8046
Iteration:  8047
Iteration:  8048
Iteration:  8049
Iteration:  8050
Iteration:  8051
Iteration:  8052
Iteration:  8053
Iteration:  8054
Iteration:  8055
Iteration:  8056
Iteration:  8057
Iteration:  8058
Iteration:  8059
Iteration:  80

Iteration:  8472
Iteration:  8473
Iteration:  8474
Iteration:  8475
Iteration:  8476
Iteration:  8477
Iteration:  8478
Iteration:  8479
Iteration:  8480
Iteration:  8481
Iteration:  8482
Iteration:  8483
Iteration:  8484
Iteration:  8485
Iteration:  8486
Iteration:  8487
Iteration:  8488
Iteration:  8489
Iteration:  8490
Iteration:  8491
Iteration:  8492
Iteration:  8493
Iteration:  8494
Iteration:  8495
Iteration:  8496
Iteration:  8497
Iteration:  8498
Iteration:  8499
Iteration:  8500
Epoch: 3/200 Iteration: 8500 Loss: 4.977974891662598
Iteration:  8501
Iteration:  8502
Iteration:  8503
Iteration:  8504
Iteration:  8505
Iteration:  8506
Iteration:  8507
Iteration:  8508
Iteration:  8509
Iteration:  8510
Iteration:  8511
Iteration:  8512
Iteration:  8513
Iteration:  8514
Iteration:  8515
Iteration:  8516
Epoch:  4
Iteration:  8517
Iteration:  8518
Iteration:  8519
Iteration:  8520
Iteration:  8521
Iteration:  8522
Iteration:  8523
Iteration:  8524
Iteration:  8525
Iteration:  8526
It

Iteration:  8938
Iteration:  8939
Iteration:  8940
Iteration:  8941
Iteration:  8942
Iteration:  8943
Iteration:  8944
Iteration:  8945
Iteration:  8946
Iteration:  8947
Iteration:  8948
Iteration:  8949
Iteration:  8950
Iteration:  8951
Iteration:  8952
Iteration:  8953
Iteration:  8954
Iteration:  8955
Iteration:  8956
Iteration:  8957
Iteration:  8958
Iteration:  8959
Iteration:  8960
Iteration:  8961
Iteration:  8962
Iteration:  8963
Iteration:  8964
Iteration:  8965
Iteration:  8966
Iteration:  8967
Iteration:  8968
Iteration:  8969
Iteration:  8970
Iteration:  8971
Iteration:  8972
Iteration:  8973
Iteration:  8974
Iteration:  8975
Iteration:  8976
Iteration:  8977
Iteration:  8978
Iteration:  8979
Iteration:  8980
Iteration:  8981
Iteration:  8982
Iteration:  8983
Iteration:  8984
Iteration:  8985
Iteration:  8986
Iteration:  8987
Iteration:  8988
Iteration:  8989
Iteration:  8990
Iteration:  8991
Iteration:  8992
Iteration:  8993
Iteration:  8994
Iteration:  8995
Iteration:  89

Iteration:  9136
Iteration:  9137
Iteration:  9138
Iteration:  9139
Iteration:  9140
Iteration:  9141
Iteration:  9142
Iteration:  9143
Iteration:  9144
Iteration:  9145
Iteration:  9146
Iteration:  9147
Iteration:  9148
Iteration:  9149
Iteration:  9150
Iteration:  9151
Iteration:  9152
Iteration:  9153
Iteration:  9154
Iteration:  9155
Iteration:  9156
Iteration:  9157
Iteration:  9158
Iteration:  9159
Iteration:  9160
Iteration:  9161
Iteration:  9162
Iteration:  9163
Iteration:  9164
Iteration:  9165
Iteration:  9166
Iteration:  9167
Iteration:  9168
Iteration:  9169
Iteration:  9170
Iteration:  9171
Iteration:  9172
Iteration:  9173
Iteration:  9174
Iteration:  9175
Iteration:  9176
Iteration:  9177
Iteration:  9178
Iteration:  9179
Iteration:  9180
Iteration:  9181
Iteration:  9182
Iteration:  9183
Iteration:  9184
Iteration:  9185
Iteration:  9186
Iteration:  9187
Iteration:  9188
Iteration:  9189
Iteration:  9190
Iteration:  9191
Iteration:  9192
Iteration:  9193
Iteration:  91